In [2]:



import os
import sam3
import torch
import torch.distributions as td
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
# from sam3.model_builder import build_sam3_video_predictor
from sam3.model_builder import build_sam3_video_model

device = torch.device("cuda")
gpus_to_use = range(torch.cuda.device_count())
checkpoint_path = "//mnt/e/weights/sam3/sam3.pt"




/home/will/code/sam3-main/sam3/model_builder.py:6: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [3]:

Ground_folder_t2='/T2SPIR/Ground'
dcm_files_t2='/T2SPIR/DICOM_anon/'
organs=[63,126,189,252]

In [4]:
patients=os.listdir('/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR_S3/')

In [5]:
patients.sort(key=lambda p: int(p))

In [6]:
sam3_model = build_sam3_video_model(checkpoint_path=checkpoint_path)
predictor = sam3_model.tracker
predictor.backbone = sam3_model.detector.backbone

INFO 2025-12-24 00:43:28,847 153007 sam3_video_base.py: 124: setting max_num_objects=10000 and num_obj_for_compile=16


In [7]:
from sam3 import build_sam3_image_model
from sam3.model.sam3_image_processor import Sam3Processor

sam3_root = os.path.join(os.path.dirname(sam3.__file__), "..")
bpe_path = f"{sam3_root}/sam3/assets/bpe_simple_vocab_16e6.txt.gz"
model_img = build_sam3_image_model(checkpoint_path=checkpoint_path,bpe_path=bpe_path, enable_inst_interactivity=True)
processor = Sam3Processor(model_img)

# **Some Function** 

In [8]:

def get_mask_all(organ,mask_dir,filenames):
    masks=[]
    for i in range(len(filenames)):
        mask=Image.open(os.path.join(mask_dir, filenames[i]))
        mask=np.asarray(mask)==organs[organ]
        masks.append(mask)
    return np.asarray(masks)

def get_input(mask_dir,filenames):
    IMGS=[]
    for i in range(len(filenames)):
        mask=Image.open(os.path.join(mask_dir, filenames[i]))
        mask=np.asarray(mask)#.convert("RGB")
        IMGS.append(mask)
    return np.asarray(IMGS)

In [9]:

def get_contour(mask_):
    contours, _ = cv2.findContours(mask_.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnt = contours[0]
    M = cv2.moments(cnt)    
    cx = int(M['m10'] / M['m00'])
    cy = int(M['m01'] / M['m00'])
    return cx,cy

def radius(center,rad,sample_rate):
    x=np.cos(np.linspace(0,2*np.pi-0.001,sample_rate))*rad
    y=np.sin(np.linspace(0,2*np.pi-0.001,sample_rate))*rad
    x=center[0]+x
    y=center[1]+y
    output=np.stack((x,y)).T
    zzz=np.sum(output>0,axis=-1)==2
    return output[np.where(zzz==1)]

def get_means(position,POS,NEG,step):
    means_p=[]
    for i in range(step):
        kkk=radius(position,POS[i],int(POS[i]*5))
        means_p.append(kkk)
    means_n=[]
    for i in range(step):
        kkk=radius(position,NEG[i],int(NEG[i]*5))
        means_n.append(kkk)
    return means_p,means_n

def Get_gmm(mean,var):
    num_of_gaus=len(mean)
    covars=torch.ones((num_of_gaus,2,2))*torch.eye(2)*var
    mean=torch.Tensor(mean)
    MVN= td.MultivariateNormal(mean,covars)
    mix=td.Categorical(torch.ones((num_of_gaus,)))
    gmm = td.MixtureSameFamily(mix, MVN)
    return gmm


def extract_mask(res_,w_,h_):
    mask_output=[]
    for i in range(len(res_)):
        for out_obj_id, out_mask in res_[i].items():
            mask_output.append(out_mask.reshape((w_,h_,1))*1.0)
    mask_output=np.asarray(mask_output).astype(np.float32)
    return mask_output

def TV_dist(img1,img2):
    tv=img2-img1
    tv2=np.sum(np.abs(tv))
    return tv,tv2

def dm_dt_func(all_masks):
    dm_dt_img=[]
    dm_dt_val=[]
    for i in range(len(all_masks)-1):
        k1,k2=TV_dist(all_masks[i],all_masks[i+1])
        dm_dt_img.append(k1)
        dm_dt_val.append(k2)
    dm_dt2_img=[]
    dm_dt2_val=[]
    for i in range(len(dm_dt_img)-1):
        k1,k2=TV_dist(dm_dt_img[i],dm_dt_img[i+1])
        dm_dt2_img.append(k1)
    for i in range(len(dm_dt_val)-1):
        dm_dt2_val.append(np.sum(np.abs(dm_dt_val[i+1]-dm_dt_val[i])))
    return dm_dt_val/np.max(dm_dt_val),dm_dt2_val/np.max(dm_dt2_val)


def dm_dx_dy_func(all_masks):
    TV_dx=[]
    TV_dy=[]
    for i in range(len(all_masks)):
        TV_dx.append(np.sum(np.abs(all_masks[i][1:,:]-all_masks[i][:-1,:])))
        TV_dy.append(np.sum(np.abs(all_masks[i][:,1:]-all_masks[i][:,:-1])))
    TV_dy=np.asarray(TV_dy)
    TV_dx=np.asarray(TV_dx)
    outs=(TV_dx+TV_dy)/2
    return outs/np.max(outs)

def metrcis_func(all_masks):
    dm_dt,dm_dt2=dm_dt_func(all_masks)
    dm_dx_dy=dm_dx_dy_func(all_masks)
    return np.sum(dm_dt),np.sum(dm_dt2),np.sum(dm_dx_dy)

def iou(outputs: np.array, labels: np.array):
    SMOOTH = 1e-6
    intersection = (outputs & labels).sum()
    union = (outputs | labels).sum()
    iou = (intersection ) / (union + SMOOTH)
    return iou

def dice(outputs: np.array, labels: np.array):
    intersection = (outputs * labels).sum()
    union = np.sum(outputs).sum()+np.sum(labels).sum()
    dice = (2*intersection ) / (union + 1e-6)
    return dice

def dice_batch(outputs: np.array, labels: np.array):
    intersection = (outputs * labels).sum((1,2))
    union = np.sum(outputs).sum((1,2))+np.sum(labels).sum((1,2))
    dice = (2*intersection ) / (union + 1e-6)
    return dice.mean()

def get_region(mask_output_,ann_frame_idx_):
    for l in range(ann_frame_idx_):
        dice_scores_=dice(mask_output_[ann_frame_idx_-l],mask_output_[ann_frame_idx_])
        if dice_scores_==0:
            left_=ann_frame_idx_-l
            break
    for r in range(ann_frame_idx_,len(mask_output_)):
        dice_scores_=dice(mask_output_[r],mask_output_[ann_frame_idx_])
        if dice_scores_==0:
            right_=r
            break
    mask_output_[:left_]=mask_output_[:left_]*0
    mask_output_[right_:]=mask_output_[right_:]*0
    return mask_output_.astype(np.int16)

def dice_batch(outputs: np.array, labels: np.array):
    outputs=np.squeeze(outputs)
    labels=np.squeeze(labels)
    intersection = (outputs * labels).sum((1,2))
    union = outputs.sum((1,2))+labels.sum((1,2))
    dice = (2*intersection + 1e-8) / (union + 1e-8)
    return dice.mean()

In [10]:

def dice_filter(results_,idx_):
    results_=np.squeeze(results_)
    imm_dim=results_.shape[1]
    final_mask=np.zeros((len(results_),imm_dim,imm_dim))

    keep=True
    for i in range(1,idx_):

        if dice(results_[idx_-i-1],results_[idx_-i])>0.5:
            keep=True
        else:
            keep=False
            final_mask[idx_-i]=results_[idx_-i]
            break
        if keep:
            final_mask[idx_-i]=results_[idx_-i]

    keep=True
    for i in range(idx_,len(results_)-1):
        
        if dice(results_[i+1],results_[i])>0.5:
            keep=True
        else:
            keep=False
            final_mask[i]=results_[i]
            break
        if keep:
            final_mask[i]=results_[i]
    return final_mask




In [11]:
def get_gmm_points(prompt,point_size,dist_pos_cata,dist_neg_cata,dist_gmm_pos,dist_gmm_neg,wid,hei):
    test_points=[]
    test_labels=[]
    if point_size==1:

        point_i=[]
        label_i=[]
        point_i.append(prompt[0])
        label_i.append(1.0)
        _dots_=dist_gmm_neg[dist_neg_cata.sample((1,)).numpy()[0]].sample((1,)).numpy()[0]
        _dots_[0]=_dots_[0]/wid
        _dots_[1]=_dots_[1]/hei
        point_i.append(_dots_)
        label_i.append(0.0)
        point_i=np.asarray(np.squeeze(point_i)).astype(np.float32)
        label_i=np.asarray(label_i).astype(np.int32)    
        test_points.append(point_i)
        test_labels.append(label_i)

    elif point_size>1:
        point_i=[]
        label_i=[]
        point_i.append(prompt[0])
        label_i.append(1.0)
        for j in range(point_size-1):
            _dots_=dist_gmm_pos[dist_pos_cata.sample((1,)).numpy()[0]].sample((1,)).numpy()[0]
            _dots_[0]=_dots_[0]/wid
            _dots_[1]=_dots_[1]/hei
            point_i.append(_dots_)
            label_i.append(1.0)
        for j in range(point_size):
            _dots_=dist_gmm_neg[dist_neg_cata.sample((1,)).numpy()[0]].sample((1,)).numpy()[0]
            _dots_[0]=_dots_[0]/wid
            _dots_[1]=_dots_[1]/hei
            point_i.append(_dots_)
            label_i.append(0.0)
        point_i=np.asarray(np.squeeze(point_i)).astype(np.float32)
        label_i=np.asarray(label_i).astype(np.int32)    
        test_points.append(point_i)
        test_labels.append(label_i)
    return test_points,test_labels

def get_score(results_,shape_,w_,h_):
    if results_==None:
        t1=1000
        t2=1000
        t3=1000
        mask_output_=np.ones(shape_)
    else:
        mask_output_=extract_mask(results_,w_,h_)
        t1,t2,t3=metrcis_func(mask_output_)
    return t1,t2,t3,mask_output_

def check_improvement(base_mask,mask_output_,k1_,k3_,t1_,t3_,mr,mr_,tres_mask):
    improvements=0
    improved=False
    if base_mask is not None:
        mask_compare=np.sum(np.abs(base_mask-mask_output_),axis=(1,2,3))

    else:
        mask_compare=np.ones((len(mask_output_)))*tres_mask
    
    imp_1=(k1_-t1_)/(k1_+1e-8) 
    imp_2=(k3_-t3_)/(k3_+1e-8)
    if imp_1>0 and imp_2>0 and mr<tres_mask:
        improvements=(imp_1 + imp_2)/2
    else:
        improvements=-np.inf 

    masks_diff_time=mask_compare>tres_mask

    add_rules=0
    if improvements>0.001 and np.sum(masks_diff_time)<=30.0 :
        print('New good points hit rule 1' )
        print('')
        print('')
        print('')
        improved=True
        add_rules=1
    elif improvements>0.1:
        print('New good points hit rule 2' )
        print('')
        print('')
        print('')
        improved=True
        add_rules=1
    elif t1_ !=1000 and (np.abs(imp_1)+np.abs(imp_2))/2<0.2:
        print('New good points hit rule 3' )
        print('')
        print('')
        print('')
        improved=True
        add_rules=1
    elif mr>tres_mask and mr_<tres_mask:
        improved=True
        print('New good points hit rule 4' )
        print('')
        print('')
        print('')
    elif mr_<mr:
        improved=True
        print('New good points hit rule 5' )
        print('')
        print('')
        print('')
    return improved,add_rules

In [12]:
def test_sam_func(pons,las,tres_,video_dir,ann_frame_idx_):
    
    inference_state = predictor.init_state(video_path=video_dir) 
    predictor.clear_all_points_in_video(inference_state)

    _, out_obj_ids, low_res_masks,out_mask_logits = predictor.add_new_points(
        inference_state=inference_state,
        frame_idx=ann_frame_idx_,
        obj_id=1,
        points=pons,
        labels=las,
        clear_old_points=False,
    )
    
    abs_mask_region=np.sum((out_mask_logits[0] > 0.0).cpu().numpy())

    if abs_mask_region>tres_:
        return None, abs_mask_region
    else:
        video_segments_f = {} 
        for out_frame_idx, out_obj_ids,low_res_masks, out_mask_logits, obj_scores  in predictor.propagate_in_video(inference_state,start_frame_idx=ann_frame_idx_, max_frame_num_to_track=240, reverse=False, propagate_preflight=True):
            video_segments_f[out_frame_idx] = {
                out_obj_id: (out_mask_logits[i] > 0.0).cpu().numpy()
                for i, out_obj_id in enumerate(out_obj_ids)
            }
        for out_frame_idx, out_obj_ids,low_res_masks, out_mask_logits, obj_scores  in predictor.propagate_in_video(inference_state,start_frame_idx=ann_frame_idx_, max_frame_num_to_track=240, reverse=True, propagate_preflight=True):
            video_segments_f[out_frame_idx] = {
                out_obj_id: (out_mask_logits[i] > 0.0).cpu().numpy()
                for i, out_obj_id in enumerate(out_obj_ids)
            }
        predictor.clear_all_points_in_video(inference_state)
        return video_segments_f, abs_mask_region

# **Result ProbSAM3**

In [13]:
Jpg_folder_save_1='/T1DUAL/Inp_InPhase_jpg'
Jpg_folder_save_2='/T1DUAL/Inp_OutPhase_jpg'    
Jpg_folder_save_3='/T2SPIR/Inp_jpg'

Ground_folder_t1='/T1DUAL/Ground'
Ground_folder_t2='/T2SPIR/Ground'


In [ ]:
def get_data_path_jpg(pat_num,path_image,path_true):
    base_path='/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR_SAM3/'
    base_path_t='/mnt/d/Dataset/dataset_sam/Chaos/Train_Sets/MR/'
    
    
    video_dirt_t1_t=base_path_t+patients[pat_num]+path_true
    video_dirt_t1_img=base_path+patients[pat_num]+path_image

    _frame_names_t1_img = [
            p for p in os.listdir(video_dirt_t1_img)
            if os.path.splitext(p)[-1] in [".jpg", ".jpeg", ".JPG", ".JPEG"]
        ]
    _frame_names_t1_img.sort(key=lambda p: int(os.path.splitext(p)[0].split('_')[-1]))

    _frame_names_t1_t = [
            p for p in os.listdir(video_dirt_t1_t)
            if os.path.splitext(p)[-1] in ['.png']#".jpg", ".jpeg", ".JPG", ".JPEG",
        ]
    # _frame_names_t1_t.sort(key=lambda p: int(os.path.splitext(p)[0].split('_')[-1]))

    path_t1=[video_dirt_t1_img , video_dirt_t1_t]
    name_t1=[_frame_names_t1_img,_frame_names_t1_t]

    return  path_t1,name_t1

In [15]:
def check_init_img(image,pons,las, mask_t):
    inference_state = processor.set_image(image)

    masks, scores, logits = model_img.predict_inst(
    inference_state,
    point_coords=pons,
    point_labels=las,
    multimask_output=True,)

    sorted_ind = np.argsort(scores)[::-1]
    masks = masks[sorted_ind]
    scores = scores[sorted_ind]
    logits = logits[sorted_ind]
    mask_p=(masks[0] > 0.0)
    return dice(np.squeeze(mask_p),np.squeeze(mask_t))

In [16]:
def __promt_idx(img_p,img_n,data_mask):
    masked_index=np.where(data_mask.sum((1,2))>0)[0].tolist()
    print(masked_index)
    ini_check=[]
    ini_check_ind=[]

    for i in range(3):
        index=masked_index.pop(np.random.randint(0,len(masked_index)))
        # print(index)
        try:
            # print('1')
            point_r,point_l=get_contour(data_mask[index])
            # print('2')
            center_point = np.array((point_r,point_l), dtype=np.float32)
            points = np.array(center_point[np.newaxis,...], dtype=np.float32)
            labels = np.array([1,], np.int32)
            # print('3')
            img_ = Image.open(img_p+'/'+img_n[index])
            zzz=check_init_img(img_,points,labels,data_mask[index])
            # print('4')
            ini_check.append(zzz)
            # print(zzz,index)
            # print('5')
            ini_check_ind.append(index)
        except:
            print("Some Error.")
    # print(ini_check_ind[np.argmax(ini_check)])
    return ini_check_ind[np.argmax(ini_check)]


In [17]:

def test_and_val_gp(input_dir,input_dir_name,imps_,masks_,_sample_=3):
    w,h=np.asarray(imps_[0]).shape[:2]
    region_cover=0.5
    Size_to_cover=(w+h)/2*region_cover
    mask_region_limit=np.asarray(w*h*region_cover).astype(np.int64)
    print('Image size is ',w*h*region_cover,mask_region_limit)
    mask_frame_idx=1000
    for i in range(10):
        try:
            mask_frame_idx=__promt_idx(input_dir,input_dir_name,masks_)
        except:
            print("Mask not right.")
        if mask_frame_idx!=1000:
            break
    # print('Prompt Index is ',mask_frame_idx)
    point_r,point_l=get_contour(masks_[mask_frame_idx])
    center_point = np.array((point_r,point_l), dtype=np.float32)
    points = np.array(center_point[np.newaxis,...], dtype=np.float32)
    labels = np.array([1,], np.int32)

    decay=0.5
    num_of_gaus=6
    variance=np.sqrt(Size_to_cover/(num_of_gaus*3))
    wave_length=Size_to_cover/num_of_gaus
    rads_pos=np.arange(1,num_of_gaus+1)*wave_length
    rads_neg=rads_pos+3*variance
    alpha=[]
    for i in range(num_of_gaus):
        alpha.append((1-decay)**i)
    alpha=np.asarray(alpha).astype(np.float32)
    beta=alpha[::-1]

    cata_alpha = td.Categorical(torch.tensor(alpha/np.sum(alpha)))
    cata_beta = td.Categorical(torch.tensor(beta/np.sum(beta)))

    dist_pos=[]
    dist_neg=[]
    means_pos,means_neg=get_means((point_r,point_l),rads_pos,rads_neg,num_of_gaus)

    for i in range(len(means_pos)):
        gmm=Get_gmm(means_pos[i],variance)
        dist_pos.append(gmm)
        gmm=Get_gmm(means_neg[i],variance)
        dist_neg.append(gmm)
    


    center_point = np.array((point_r/w,point_l/h), dtype=np.float32)
    points = np.array(center_point[np.newaxis,...], dtype=np.float32)
    labels = np.array([1,], np.int32)

    points_tensor = torch.tensor(points, dtype=torch.float32)
    points_labels_tensor = torch.tensor(labels, dtype=torch.int32)  
    
    
    results,mr_original=test_sam_func(points_tensor,points_labels_tensor,w*h,input_dir,mask_frame_idx)
    mask_output=extract_mask(results,w,h)
    filtered_output=dice_filter(mask_output,mask_frame_idx)


    # # print(mask_output[0])

    mask_output_shape=mask_output.shape
    k1,k2,k3=metrcis_func(mask_output)
    print('           Initial metrics:                   ',k1,k2,k3,mr_original)
    
    mask_candidate=[]
    mask_candidate.append(dice_batch(mask_output,masks_))
    print('Initial dice score is ',mask_candidate)

    for i in range(_sample_):
        test_points,test_labels=get_gmm_points(points,1,cata_alpha,cata_beta,dist_pos,dist_neg,w,h)
        points_tensor = torch.tensor(test_points, dtype=torch.float32)
        points_labels_tensor = torch.tensor(test_labels, dtype=torch.int32)  
        results_test,mr_test=test_sam_func(points_tensor,points_labels_tensor,w*h,input_dir,mask_frame_idx) 
        t1,t2,t3,mask_output_=get_score(results_test,mask_output_shape,w,h)     
        improved,add_rules=check_improvement(mask_output,mask_output_,k1,k3,t1,t3,mr_original,mr_test,mask_region_limit)
        print(t1,t2,t3,mr_test)
        if improved:
            k1=t1
            k3=t3
            filtered_output=dice_filter(mask_output_,mask_frame_idx)
            mask_candidate.append(dice_batch(filtered_output,masks_))
    for j in (2,3):
        for i in range(_sample_):
            test_points,test_labels=get_gmm_points(points,j,cata_alpha,cata_beta,dist_pos,dist_neg,w,h)
            points_tensor = torch.tensor(test_points, dtype=torch.float32)
            points_labels_tensor = torch.tensor(test_labels, dtype=torch.int32)  
            results_test,mr_test=test_sam_func(points_tensor,points_labels_tensor,w*h,input_dir,mask_frame_idx) 
            t1,t2,t3,mask_output_=get_score(results_test,mask_output_shape,w,h)     
            print(t1,t2,t3,mr_test)
            improved,add_rules=check_improvement(mask_output,mask_output_,k1,k3,t1,t3,mr_original,mr_test,mask_region_limit)
            if improved :
                k1=t1
                k3=t3
                filtered_output=dice_filter(mask_output_,mask_frame_idx)
                mask_candidate.append(dice_batch(filtered_output,masks_))

    return np.max(mask_candidate)



In [18]:
reults_organ_0=[]
for i in range(len(patients)):
    f1,f2=get_data_path_jpg(i,Jpg_folder_save_2,Ground_folder_t1)
    mask_os=get_mask_all(0,f1[1],f2[1])
    input_imgaes=get_input(f1[0],f2[0])
    if mask_os.shape[1] != input_imgaes.shape[1]:
        mask_os.resize((input_imgaes.shape[0],input_imgaes.shape[1],input_imgaes.shape[2]))
    kkk=test_and_val_gp(f1[0],f2[0],input_imgaes,mask_os,_sample_=3)
    reults_organ_0.append(kkk)
print(reults_organ_0)
print(np.mean(reults_organ_0))

reults_organ_0_1=[]
for i in range(len(patients)):
    f1,f2=get_data_path_jpg(i,Jpg_folder_save_3,Ground_folder_t2)
    mask_os=get_mask_all(0,f1[1],f2[1])
    input_imgaes=get_input(f1[0],f2[0])
    if mask_os.shape[1] != input_imgaes.shape[1]:
        mask_os.resize((input_imgaes.shape[0],input_imgaes.shape[1],input_imgaes.shape[2]))
    kkk=test_and_val_gp(f1[0],f2[0],input_imgaes,mask_os,_sample_=3)
    reults_organ_0_1.append(kkk)
print(reults_organ_0_1)
print(np.mean(reults_organ_0_1))

Image size is  32768.0 32768
[10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34]


frame loading (JPEG): 100%|██████████| 35/35 [00:00<00:00, 61.98it/s]


propagate in video:   0%|          | 0/1 [00:00<?, ?it/s]

propagate in video:   0%|          | 0/35 [00:00<?, ?it/s]

/tmp/ipykernel_153007/3357707488.py:73: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  points_tensor = torch.tensor(test_points, dtype=torch.float32)


...

10.9732 6.603057 7.326812 6392
New good points hit rule 3





frame loading (JPEG): 100%|██████████| 26/26 [00:00<00:00, 64.04it/s]


propagate in video:   0%|          | 0/21 [00:00<?, ?it/s]

propagate in video:   0%|          | 0/6 [00:00<?, ?it/s]

12.219881 5.495838 14.090197 1318
[0.9377552633852891, 0.8617757796754184, 0.9321078034036361, 0.9112401316523069, 0.917597174404987, 0.5764172699398481, 0.9186172335873927, 0.6600063349483913, 0.7333827832647895, 0.9405333793051649, 0.8947301951854076, 0.9771438943279722, 0.9530517606431279, 0.9375573099277097, 0.9436896563160705, 0.8750206596281, 0.9051962425166997, 0.9004785542083409, 0.9444751588983911, 0.9464845856292665]
0.8833630585424157


In [21]:
reults_organ_1=[]
for i in range(len(patients)):
    f1,f2=get_data_path_jpg(i,Jpg_folder_save_2,Ground_folder_t1)
    mask_os=get_mask_all(1,f1[1],f2[1])
    input_imgaes=get_input(f1[0],f2[0])
    if mask_os.shape[1] != input_imgaes.shape[1]:
        mask_os.resize((input_imgaes.shape[0],input_imgaes.shape[1],input_imgaes.shape[2]))
    kkk=test_and_val_gp(f1[0],f2[0],input_imgaes,mask_os,_sample_=3)
    reults_organ_1.append(kkk)
print(reults_organ_1)
print(np.mean(reults_organ_1))

reults_organ_1_1=[]
for i in range(len(patients)):
    f1,f2=get_data_path_jpg(i,Jpg_folder_save_3,Ground_folder_t2)
    mask_os=get_mask_all(1,f1[1],f2[1])
    input_imgaes=get_input(f1[0],f2[0])
    if mask_os.shape[1] != input_imgaes.shape[1]:
        mask_os.resize((input_imgaes.shape[0],input_imgaes.shape[1],input_imgaes.shape[2]))
    kkk=test_and_val_gp(f1[0],f2[0],input_imgaes,mask_os,_sample_=3)
    reults_organ_1_1.append(kkk)
print(reults_organ_1_1)
print(np.mean(reults_organ_1_1))

Image size is  32768.0 32768
[10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]


frame loading (JPEG): 100%|██████████| 35/35 [00:00<00:00, 65.56it/s]


propagate in video:   0%|          | 0/19 [00:00<?, ?it/s]

propagate in video:   0%|          | 0/17 [00:00<?, ?it/s]

           Initial metrics:                    10.385475 6.1590104 22.900002 762
Initial dice score is  [0.6265674532756931]


...

12.48736 5.7388062 14.338989 15209


frame loading (JPEG): 100%|██████████| 26/26 [00:00<00:00, 63.26it/s]


propagate in video:   0%|          | 0/15 [00:00<?, ?it/s]

propagate in video:   0%|          | 0/12 [00:00<?, ?it/s]

7.7022862 7.219925 8.420153 15394
[0.9809098793710042, 0.9696106045941667, 0.9523475565885755, 0.9459307501927424, 0.9078858639058729, 0.982093779847559, 0.9489818945396303, 0.9470087027456212, 0.9784264674699409, 0.9361536236853667, 0.9223107766799643, 0.9373772409455654, 0.9451512305214274, 0.9579759867175945, 0.926842081275205, 0.9350367567983849, 0.9660743064272693, 0.9763151931956922, 0.957002575275528, 0.9735216986851289]
0.952347848473112


In [22]:
reults_organ_2=[]
for i in range(len(patients)):
    f1,f2=get_data_path_jpg(i,Jpg_folder_save_2,Ground_folder_t1)
    mask_os=get_mask_all(2,f1[1],f2[1])
    input_imgaes=get_input(f1[0],f2[0])
    if mask_os.shape[1] != input_imgaes.shape[1]:
        mask_os.resize((input_imgaes.shape[0],input_imgaes.shape[1],input_imgaes.shape[2]))
    kkk=test_and_val_gp(f1[0],f2[0],input_imgaes,mask_os,_sample_=3)
    reults_organ_2.append(kkk)
print(reults_organ_2)
print(np.mean(reults_organ_2))

reults_organ_2_1=[]
for i in range(len(patients)):
    f1,f2=get_data_path_jpg(i,Jpg_folder_save_3,Ground_folder_t2)
    mask_os=get_mask_all(2,f1[1],f2[1])
    input_imgaes=get_input(f1[0],f2[0])
    if mask_os.shape[1] != input_imgaes.shape[1]:
        mask_os.resize((input_imgaes.shape[0],input_imgaes.shape[1],input_imgaes.shape[2]))
    kkk=test_and_val_gp(f1[0],f2[0],input_imgaes,mask_os,_sample_=3)
    reults_organ_2_1.append(kkk)
print(reults_organ_2_1)
print(np.mean(reults_organ_2_1))

Image size is  32768.0 32768
[12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]


frame loading (JPEG): 100%|██████████| 35/35 [00:00<00:00, 64.05it/s]


propagate in video:   0%|          | 0/11 [00:00<?, ?it/s]

propagate in video:   0%|          | 0/25 [00:00<?, ?it/s]

           Initial metrics:                    12.243363 5.5885415 20.957146 747
Initial dice score is  [0.6366137843283162]


...

10.125 5.75076 8.452283 1545


frame loading (JPEG): 100%|██████████| 26/26 [00:00<00:00, 59.83it/s]


propagate in video:   0%|          | 0/14 [00:00<?, ?it/s]

propagate in video:   0%|          | 0/13 [00:00<?, ?it/s]

6.9329166 6.1023006 10.52729 8039
[0.973721796880214, 0.968951384916567, 0.9491859945109179, 0.9461923286201592, 0.8741382579186812, 0.9809166677100468, 0.9465623139534824, 0.9328079883500077, 0.9760975000760229, 0.815830688924531, 0.9331169201633068, 0.9248417422200953, 0.9739757087646882, 0.9169365059561775, 0.9654461822664456, 0.9145932905846893, 0.9083723817426292, 0.9705388691665877, 0.9574571359710421, 0.7430525681620807]
0.9286368113429185


In [23]:
reults_organ_3=[]
for i in range(len(patients)):
    f1,f2=get_data_path_jpg(i,Jpg_folder_save_2,Ground_folder_t1)
    mask_os=get_mask_all(3,f1[1],f2[1])
    input_imgaes=get_input(f1[0],f2[0])
    if mask_os.shape[1] != input_imgaes.shape[1]:
        mask_os.resize((input_imgaes.shape[0],input_imgaes.shape[1],input_imgaes.shape[2]))
    kkk=test_and_val_gp(f1[0],f2[0],input_imgaes,mask_os,_sample_=3)
    reults_organ_3.append(kkk)
print(reults_organ_3)
print(np.mean(reults_organ_3))

reults_organ_3_1=[]
for i in range(len(patients)):
    f1,f2=get_data_path_jpg(i,Jpg_folder_save_3,Ground_folder_t2)
    mask_os=get_mask_all(3,f1[1],f2[1])
    input_imgaes=get_input(f1[0],f2[0])
    if mask_os.shape[1] != input_imgaes.shape[1]:
        mask_os.resize((input_imgaes.shape[0],input_imgaes.shape[1],input_imgaes.shape[2]))
    kkk=test_and_val_gp(f1[0],f2[0],input_imgaes,mask_os,_sample_=3)
    reults_organ_3_1.append(kkk)
print(reults_organ_3_1)
print(np.mean(reults_organ_3_1))

Image size is  32768.0 32768
[17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34]


frame loading (JPEG): 100%|██████████| 35/35 [00:00<00:00, 61.06it/s]


propagate in video:   0%|          | 0/5 [00:00<?, ?it/s]

propagate in video:   0%|          | 0/31 [00:00<?, ?it/s]

           Initial metrics:                    4.6296296 3.9076517 17.33846 515
Initial dice score is  [0.7701461760493054]


...

3.6522732 2.883226 3.698959 2371
New good points hit rule 1





frame loading (JPEG): 100%|██████████| 26/26 [00:00<00:00, 64.45it/s]


propagate in video:   0%|          | 0/10 [00:00<?, ?it/s]

propagate in video:   0%|          | 0/17 [00:00<?, ?it/s]

8.7283745 5.3846827 16.591974 11488
[0.9534678303868416, 0.907435031007693, 0.9608596214281235, 0.9429184572958721, 0.9787979303735099, 0.9161104016263538, 0.9146201004659729, 0.7022173926345103, 0.9417179586269767, 0.8433488401394571, 0.9169123035043742, 0.9799004454232776, 0.9103244585518473, 0.9647340482888794, 0.9823320787917104, 0.9416132387902275, 0.9855634571049035, 0.7134333298478042, 0.9035541651665335, 0.9266861365999051]
0.9143273613027386


In [24]:
print('organ',' liver: ',(np.mean(reults_organ_0_1)+np.mean(reults_organ_0))/2)
print('organ',' right kidney: ',(np.mean(reults_organ_1_1)+np.mean(reults_organ_1))/2)
print('organ',' left kidney: ',(np.mean(reults_organ_2_1)+np.mean(reults_organ_2))/2)
print('organ',' spleen: ',(np.mean(reults_organ_3_1)+np.mean(reults_organ_3))/2)

organ  liver:  0.8985926873640342
organ  right kidney:  0.9469483546550501
organ  left kidney:  0.9289438427606855
organ  spleen:  0.919870880727184


In [25]:
np.mean([91.99, 89.85, 92.89, 94.69, 90.65])

92.01399999999998

# **Baseline**

In [26]:

def test_and_val_baseline(input_dir,input_dir_name,imps_,masks_,):
    w,h=np.asarray(imps_[0]).shape[:2]
    region_cover=0.5
    Size_to_cover=(w+h)/2*region_cover
    mask_region_limit=np.asarray(w*h*region_cover).astype(np.int64)
    print('Image size is ',w*h*region_cover,mask_region_limit)
    mask_frame_idx=1000
    for i in range(10):
        try:
            mask_frame_idx=__promt_idx(input_dir,input_dir_name,masks_)
        except:
            print("Mask not right.")
        if mask_frame_idx!=1000:
            break
    # print('Prompt Index is ',mask_frame_idx)
    point_r,point_l=get_contour(masks_[mask_frame_idx])
    center_point = np.array((point_r,point_l), dtype=np.float32)
    points = np.array(center_point[np.newaxis,...], dtype=np.float32)
    labels = np.array([1,], np.int32)



    center_point = np.array((point_r/w,point_l/h), dtype=np.float32)
    points = np.array(center_point[np.newaxis,...], dtype=np.float32)
    labels = np.array([1,], np.int32)

    points_tensor = torch.tensor(points, dtype=torch.float32)
    points_labels_tensor = torch.tensor(labels, dtype=torch.int32)  
    
    
    results,mr_original=test_sam_func(points_tensor,points_labels_tensor,w*h,input_dir,mask_frame_idx)
    mask_output=extract_mask(results,w,h)
    # filtered_output=dice_filter(mask_output,mask_frame_idx)


    # # print(mask_output[0])

    mask_output_shape=mask_output.shape
    k1,k2,k3=metrcis_func(mask_output)
    print('           Initial metrics:                   ',k1,k2,k3,mr_original)
    
    mask_candidate=[]
    mask_candidate.append(dice_batch(mask_output,masks_))
    print('Initial dice score is ',mask_candidate)

    return np.max(mask_candidate)



In [28]:
baseline_reults_organ_0=[]
for i in range(len(patients)):
    f1,f2=get_data_path_jpg(i,Jpg_folder_save_2,Ground_folder_t1)
    mask_os=get_mask_all(0,f1[1],f2[1])
    input_imgaes=get_input(f1[0],f2[0])
    if mask_os.shape[1] != input_imgaes.shape[1]:
        mask_os.resize((input_imgaes.shape[0],input_imgaes.shape[1],input_imgaes.shape[2]))
    kkk=test_and_val_baseline(f1[0],f2[0],input_imgaes,mask_os)
    baseline_reults_organ_0.append(kkk)
print(baseline_reults_organ_0)
print(np.mean(baseline_reults_organ_0))

baseline_reults_organ_0_1=[]
for i in range(len(patients)):
    f1,f2=get_data_path_jpg(i,Jpg_folder_save_3,Ground_folder_t2)
    mask_os=get_mask_all(0,f1[1],f2[1])
    input_imgaes=get_input(f1[0],f2[0])
    if mask_os.shape[1] != input_imgaes.shape[1]:
        mask_os.resize((input_imgaes.shape[0],input_imgaes.shape[1],input_imgaes.shape[2]))
    kkk=test_and_val_baseline(f1[0],f2[0],input_imgaes,mask_os)
    baseline_reults_organ_0_1.append(kkk)
print(baseline_reults_organ_0_1)
print(np.mean(baseline_reults_organ_0_1))

Image size is  32768.0 32768
[10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34]


frame loading (JPEG): 100%|██████████| 35/35 [00:00<00:00, 66.27it/s]


propagate in video:   0%|          | 0/19 [00:00<?, ?it/s]

propagate in video:   0%|          | 0/17 [00:00<?, ?it/s]

           Initial metrics:                    5.3821526 3.453859 13.761316 1017
Initial dice score is  [0.9036706125612153]
Image size is  41472.0 41472
[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]


...

           Initial metrics:                    11.717845 7.9518504 16.19256 8988
Initial dice score is  [0.5796858482836348]
Image size is  32768.0 32768
[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]


frame loading (JPEG): 100%|██████████| 26/26 [00:00<00:00, 64.66it/s]


propagate in video:   0%|          | 0/10 [00:00<?, ?it/s]

propagate in video:   0%|          | 0/17 [00:00<?, ?it/s]

           Initial metrics:                    9.858497 8.663451 12.570175 4612
Initial dice score is  [0.8529923238333359]
[0.8797687378055813, 0.2027927589134349, 0.397543917783238, 0.20523452097873654, 0.8751709012714346, 0.5764172699398481, 0.5585868399812614, 0.2067503726021625, 0.7331008957766354, 0.623535304558253, 0.5085568078855854, 0.6594513691568511, 0.9606507949281295, 0.7161477247900994, 0.9362736138361147, 0.7462389066574775, 0.573661353560941, 0.4312257850040796, 0.5796858482836348, 0.8529923238333359]
0.6111893023773417


In [ ]:
baseline_reults_organ_1=[]
for i in range(len(patients)):
    f1,f2=get_data_path_jpg(i,Jpg_folder_save_2,Ground_folder_t1)
    mask_os=get_mask_all(1,f1[1],f2[1])
    input_imgaes=get_input(f1[0],f2[0])
    if mask_os.shape[1] != input_imgaes.shape[1]:
        mask_os.resize((input_imgaes.shape[0],input_imgaes.shape[1],input_imgaes.shape[2]))
    kkk=test_and_val_baseline(f1[0],f2[0],input_imgaes,mask_os)
    baseline_reults_organ_1.append(kkk)
print(baseline_reults_organ_1)
print(np.mean(baseline_reults_organ_1))

baseline_reults_organ_1_1=[]
for i in range(len(patients)):
    f1,f2=get_data_path_jpg(i,Jpg_folder_save_3,Ground_folder_t2)
    mask_os=get_mask_all(1,f1[1],f2[1])
    input_imgaes=get_input(f1[0],f2[0])
    if mask_os.shape[1] != input_imgaes.shape[1]:
        mask_os.resize((input_imgaes.shape[0],input_imgaes.shape[1],input_imgaes.shape[2]))
    kkk=test_and_val_baseline(f1[0],f2[0],input_imgaes,mask_os)
    baseline_reults_organ_1_1.append(kkk)
print(baseline_reults_organ_1_1)
print(np.mean(baseline_reults_organ_1_1))

In [29]:
print('organ',' liver: ',(np.mean(baseline_reults_organ_0_1)+np.mean(baseline_reults_organ_0))/2)

organ  liver:  0.6989797914611313


In [30]:
baseline_reults_organ_1=[]
for i in range(len(patients)):
    f1,f2=get_data_path_jpg(i,Jpg_folder_save_2,Ground_folder_t1)
    mask_os=get_mask_all(1,f1[1],f2[1])
    input_imgaes=get_input(f1[0],f2[0])
    if mask_os.shape[1] != input_imgaes.shape[1]:
        mask_os.resize((input_imgaes.shape[0],input_imgaes.shape[1],input_imgaes.shape[2]))
    kkk=test_and_val_baseline(f1[0],f2[0],input_imgaes,mask_os)
    baseline_reults_organ_1.append(kkk)
print(baseline_reults_organ_1)
print(np.mean(baseline_reults_organ_1))

baseline_reults_organ_1_1=[]
for i in range(len(patients)):
    f1,f2=get_data_path_jpg(i,Jpg_folder_save_3,Ground_folder_t2)
    mask_os=get_mask_all(1,f1[1],f2[1])
    input_imgaes=get_input(f1[0],f2[0])
    if mask_os.shape[1] != input_imgaes.shape[1]:
        mask_os.resize((input_imgaes.shape[0],input_imgaes.shape[1],input_imgaes.shape[2]))
    kkk=test_and_val_baseline(f1[0],f2[0],input_imgaes,mask_os)
    baseline_reults_organ_1_1.append(kkk)
print(baseline_reults_organ_1_1)
print(np.mean(baseline_reults_organ_1_1))

Image size is  32768.0 32768
[10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]


frame loading (JPEG): 100%|██████████| 35/35 [00:00<00:00, 64.86it/s]


propagate in video:   0%|          | 0/13 [00:00<?, ?it/s]

propagate in video:   0%|          | 0/23 [00:00<?, ?it/s]

           Initial metrics:                    10.090395 5.8345323 22.89394 722
Initial dice score is  [0.6470526971886528]
Image size is  41472.0 41472
[2, 3, 4, 5, 6, 7, 8, 9, 10, 11]


...

           Initial metrics:                    7.655058 6.271676 9.210526 1179
Initial dice score is  [0.8906467587034911]
Image size is  32768.0 32768
[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]


frame loading (JPEG): 100%|██████████| 26/26 [00:00<00:00, 64.62it/s]


propagate in video:   0%|          | 0/14 [00:00<?, ?it/s]

propagate in video:   0%|          | 0/13 [00:00<?, ?it/s]

           Initial metrics:                    8.767676 2.9494948 9.735294 796
Initial dice score is  [0.9744660387354591]
[0.7837200688210888, 0.9693299426348232, 0.9795392499200931, 0.913308690682431, 0.8965513534672795, 0.869518659620288, 0.9130987346166459, 0.9469290863355162, 0.9468424328506533, 0.9382914741947034, 0.7271010379074078, 0.8435939233092978, 0.9449238286608648, 0.7498667254338459, 0.9222410592423024, 0.9276924951080243, 0.755186077151852, 0.7524387889638969, 0.8906467587034911, 0.9744660387354591]
0.8822643213179981


In [31]:
baseline_reults_organ_2=[]
for i in range(len(patients)):
    f1,f2=get_data_path_jpg(i,Jpg_folder_save_2,Ground_folder_t1)
    mask_os=get_mask_all(2,f1[1],f2[1])
    input_imgaes=get_input(f1[0],f2[0])
    if mask_os.shape[1] != input_imgaes.shape[1]:
        mask_os.resize((input_imgaes.shape[0],input_imgaes.shape[1],input_imgaes.shape[2]))
    kkk=test_and_val_baseline(f1[0],f2[0],input_imgaes,mask_os)
    baseline_reults_organ_2.append(kkk)
print(baseline_reults_organ_2)
print(np.mean(baseline_reults_organ_2))

baseline_reults_organ_2_1=[]
for i in range(len(patients)):
    f1,f2=get_data_path_jpg(i,Jpg_folder_save_3,Ground_folder_t2)
    mask_os=get_mask_all(2,f1[1],f2[1])
    input_imgaes=get_input(f1[0],f2[0])
    if mask_os.shape[1] != input_imgaes.shape[1]:
        mask_os.resize((input_imgaes.shape[0],input_imgaes.shape[1],input_imgaes.shape[2]))
    kkk=test_and_val_baseline(f1[0],f2[0],input_imgaes,mask_os)
    baseline_reults_organ_2_1.append(kkk)
print(baseline_reults_organ_2_1)
print(np.mean(baseline_reults_organ_2_1))

Image size is  32768.0 32768
[12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]


frame loading (JPEG): 100%|██████████| 35/35 [00:00<00:00, 65.42it/s]


propagate in video:   0%|          | 0/11 [00:00<?, ?it/s]

propagate in video:   0%|          | 0/25 [00:00<?, ?it/s]

           Initial metrics:                    12.243363 5.5885415 20.957146 747
Initial dice score is  [0.6366137843283162]
Image size is  41472.0 41472
[3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


...

           Initial metrics:                    7.293065 3.5078301 9.935898 980
Initial dice score is  [0.956116489980508]
Image size is  32768.0 32768
[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]


frame loading (JPEG): 100%|██████████| 26/26 [00:00<00:00, 64.04it/s]


propagate in video:   0%|          | 0/14 [00:00<?, ?it/s]

propagate in video:   0%|          | 0/13 [00:00<?, ?it/s]

           Initial metrics:                    11.43007 6.4333334 13.145833 835
Initial dice score is  [0.510535577229774]
[0.6398724210729736, 0.8933395925774614, 0.8497618083427567, 0.7076632225738978, 0.3876041449111533, 0.8418969270121927, 0.7790084594436242, 0.8579009677277148, 0.9753250846735159, 0.9282698446585097, 0.7689024859435266, 0.9079325823528301, 0.7958620957165883, 0.6837799857048057, 0.8925482467487309, 0.9145932905846893, 0.8507610862614916, 0.46549728989683126, 0.956116489980508, 0.510535577229774]
0.7803585801706787


In [32]:
baseline_reults_organ_3=[]
for i in range(len(patients)):
    f1,f2=get_data_path_jpg(i,Jpg_folder_save_2,Ground_folder_t1)
    mask_os=get_mask_all(3,f1[1],f2[1])
    input_imgaes=get_input(f1[0],f2[0])
    if mask_os.shape[1] != input_imgaes.shape[1]:
        mask_os.resize((input_imgaes.shape[0],input_imgaes.shape[1],input_imgaes.shape[2]))
    kkk=test_and_val_baseline(f1[0],f2[0],input_imgaes,mask_os)
    baseline_reults_organ_3.append(kkk)
print(baseline_reults_organ_3)
print(np.mean(baseline_reults_organ_3))

baseline_reults_organ_3_1=[]
for i in range(len(patients)):
    f1,f2=get_data_path_jpg(i,Jpg_folder_save_3,Ground_folder_t2)
    mask_os=get_mask_all(3,f1[1],f2[1])
    input_imgaes=get_input(f1[0],f2[0])
    if mask_os.shape[1] != input_imgaes.shape[1]:
        mask_os.resize((input_imgaes.shape[0],input_imgaes.shape[1],input_imgaes.shape[2]))
    kkk=test_and_val_baseline(f1[0],f2[0],input_imgaes,mask_os)
    baseline_reults_organ_3_1.append(kkk)
print(baseline_reults_organ_3_1)
print(np.mean(baseline_reults_organ_3_1))

Image size is  32768.0 32768
[17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34]


frame loading (JPEG): 100%|██████████| 35/35 [00:00<00:00, 65.37it/s]


propagate in video:   0%|          | 0/1 [00:00<?, ?it/s]

propagate in video:   0%|          | 0/35 [00:00<?, ?it/s]

           Initial metrics:                    5.2322464 3.2072937 8.795621 274
Initial dice score is  [0.8002638097810197]
Image size is  41472.0 41472
[10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


...

           Initial metrics:                    5.780488 3.8078008 16.0625 1833
Initial dice score is  [0.44137053065914034]
Image size is  32768.0 32768
[10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]


frame loading (JPEG): 100%|██████████| 26/26 [00:00<00:00, 64.22it/s]


propagate in video:   0%|          | 0/14 [00:00<?, ?it/s]

propagate in video:   0%|          | 0/13 [00:00<?, ?it/s]

           Initial metrics:                    8.8027525 6.625392 12.603053 691
Initial dice score is  [0.5847731759907533]
[0.6443177980210257, 0.5978168534556473, 0.4326619461094806, 0.5350964075279939, 0.9474858101299828, 0.5831282106227259, 0.5803215888524169, 0.7925674922047536, 0.4946893264189265, 0.4820640434664841, 0.6406967515041312, 0.5194062382835309, 0.45248196492266435, 0.8823896423613544, 0.6691272502383184, 0.7431169758940593, 0.45420304719553745, 0.2871280115939436, 0.44137053065914034, 0.5847731759907533]
0.5882421532726434


In [33]:
print('organ',' liver: ',(np.mean(baseline_reults_organ_0_1)+np.mean(baseline_reults_organ_0))/2)
print('organ',' right kidney: ',(np.mean(baseline_reults_organ_1_1)+np.mean(baseline_reults_organ_1))/2)
print('organ',' left kidney: ',(np.mean(baseline_reults_organ_2_1)+np.mean(baseline_reults_organ_2))/2)
print('organ',' spleen: ',(np.mean(baseline_reults_organ_3_1)+np.mean(baseline_reults_organ_3))/2)

organ  liver:  0.6989797914611313
organ  right kidney:  0.8441663637810204
organ  left kidney:  0.7753774603207716
organ  spleen:  0.6804572845287679


In [34]:
np.mean([68.04, 69.89, 77.53, 84.41])

74.9675